# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    G = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter = '\t')
    
    return G

In [3]:
#answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [4]:
def answer_two():
    
    G = answer_one()
    nx.set_node_attributes(G, 'type', {i : 'employee' for i in employees})
    nx.set_node_attributes(G, 'type', {i : 'movie' for i in movies})
    
    return G

In [5]:
#answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [6]:
def answer_three():
    
    G = answer_two()
    
    return bipartite.weighted_projected_graph(G, employees)

In [7]:
#answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [8]:
def answer_four():
        
    G = nx.read_edgelist('Employee_Relationships.txt', data = [('relationship', int)])

    df1 = pd.DataFrame(G.edges(data=True), columns=['employee1', 'employee2', 'relationship'])
    df1['relationship'] = df1['relationship'].apply(lambda x : x['relationship'])

    df2 = pd.DataFrame(answer_three().edges(data = True), columns=['employee1', 'employee2', 'common_movies'])
    df2['common_movies'] = df2['common_movies'].apply(lambda x : x['weight'])
    def f(x):
        if x['employee1'][0] > x['employee2']:
            return x['employee2'] + x['employee1']
        else :
            return x['employee1'] + x['employee2']
    df1['pair'] = df1.apply(f, axis = 1)
    df2['pair'] = df2.apply(f, axis = 1)
    df = pd.merge(df1.drop(['employee1', 'employee2'], axis = 1), df2.drop(['employee1', 'employee2'], axis = 1),
              how='left', on=['pair'])
    df['common_movies'][(df['common_movies'].isnull())] = df[(df['common_movies'].isnull())]['common_movies'].apply(lambda x : 0)
    
    return df.corr()['relationship'][1]

In [9]:
#answer_four()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.78839622217334759